<a href="https://colab.research.google.com/github/sonialouise/ts_class/blob/main/notebooks/SpontaneousActivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spontaneous Neuronal Activity Classifier
This notebook allows users to upload data in order to run a pretrained model to identify neurons exhibiting spontaneous activity.

In [ ]:
#@markdown The Github repository with all custom functions, the ground truth datasets and the pretrained models is copied to the environment of this notebook. This will take a couple of seconds.

#@markdown *Note: You can check the code underlying each cell by double-clicking on it.*

import os

# If in Colab and not yet downloaded, download GitHub repository and change working directory
if os.getcwd() == '/content':  
    !git clone https://github.com/sonialouise/ts_class.git
    os.chdir('ts_class')
    
# If executed as jupyter notebook on own computer, change to parent directory for imports
if os.path.basename( os.getcwd() ) == 'activity_classifier':
    %cd ..
    print('New working directory:', os.getcwd() )

fatal: destination path 'ts_class' already exists and is not an empty directory.


In [ ]:
!git pull 

Already up to date.


In [ ]:
#@markdown Downloads packages from public repository, and packages from ts_class

%%capture
!pip install -r requirements.txt

# python imports
import pickle
import argparse
import logging
import pandas as pd

# ts_class packages, imported from the downloaded Github repository
from activity_classifier.main import run_model
from activity_classifier.config import TSF_MODEL, RISE_MODEL, OBS, PREDICTION, OUTPUT_PATH
from activity_classifier.prepare_data import prepare_data
from activity_classifier.retrain_models import retrain_tsf, retrain_rise

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

In [ ]:
#@markdown If you are testing the script, you can leave everything unchanged. If you want to apply the algorithm to your own data, you have to upload your data first. The paragraph above tells you how to format and name the file. You can do this by clicking on the **folder symbol ("Files")** on the left side of the Colaboratory notebook. Next, indicate the path of the uploaded file in the variable **`example_file`**. Finally, indicate the sampling rate of your recordings in the variable **`frame_rate`**.

example_file = "data/test_set.csv" #@param {type:"string"}

frame_no = 550 #@param {type:"number"}

# try:

#   traces = load_neurons_x_time( example_file )
#   print('Number of neurons in dataset:', traces.shape[0])
#   print('Number of timepoints in dataset:', traces.shape[1])

# except Exception as e:
  
#   print('\nSomething went wrong!\nEither the target file is missing, in this case please provide the correct location.\nOr your file is not yet completely uploaded, in this case wait until the upload is completed.\n')

#   print('Error message: '+str(e))
  

In [ ]:
    #@markdown If your data contains more then 550 frames, you will need to retrain the models - Play this cell
    
    logging.info("1. Reading csv file...")
    data = pd.read_csv('data/training_data.csv', header=0)
    logging.info("2. Normalising data...")
    data = pd.concat([prepare_data(data.iloc[:, 0:frame_no]), data['status']], axis=1)
    logging.info("3. Retraining Time Series Classifier...")
    retrain_tsf(data)
    logging.info("4. Retraining Random Interval Spectral Ensemble...")
    retrain_rise(data)
    logging.info("5. Retraining Complete")

RESULTS TSF: {'ACCURACY': 0.8821812596006143, 'PRECISION': 0.8653881480312476, 'RECALL': 0.8294300144300145}
RESULTS RISE: {'ACCURACY': 0.9205837173579109, 'PRECISION': 0.9153151598803774, 'RECALL': 0.8799477495129668}


In [ ]:
#@markdown Now we can run the model on your data

logging.info("1. Reading csv file...")
data = pd.read_csv(example_file, header=0)
logging.info("2. Normalising data...")
data = prepare_data(data.iloc[:, 0:int(frame_no)])
logging.info("3. Predicting with TimeSeries Forest Classifier...")
data = run_model(data, TSF_MODEL, 'TSF')
logging.info("4. Predicting with Random Interval Spectral Ensemble...")
data = run_model(data, RISE_MODEL, 'RISE')
logging.info("5. Saving output...")
data.to_csv(OUTPUT_PATH)
logging.info("6. Process complete")

/content/ts_class/activity_classifier/prepare_data.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[OBS] = [pd.Series(row) for row in np.array(data)]
